In [2]:
# Evaluation - Teil 4
# Leistungsbeurteilung 259 - Maschinelles Lernen

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, classification_report
import warnings
warnings.filterwarnings('ignore')

# Daten laden (passt den Dateinamen an deinen an!)
df = pd.read_csv('zugzahlen_pro_monat.csv', delimiter=';', encoding='utf-8')

# Daten vorbereiten
df['bezugsmonat_dt'] = pd.to_datetime(df['bezugsmonat'])
df['Jahr'] = df['bezugsmonat_dt'].dt.year  
df['Monat'] = df['bezugsmonat_dt'].dt.month

# One-Hot-Encoding
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_strecke = ohe.fit_transform(df[['Strecke_Bezeichnung', 'Abschnitt']])
encoded_columns = ohe.get_feature_names_out(['Strecke_Bezeichnung', 'Abschnitt'])
df_encoded = pd.DataFrame(encoded_strecke, columns=encoded_columns)

# Features zusammenführen
X = pd.concat([df_encoded, df[['Jahr', 'Monat']]], axis=1)
y = pd.to_numeric(df['DTV_Bezugsmonat'], errors='coerce')

# Fehlende Werte entfernen
mask = y.notna()
X = X[mask]
y = y[mask]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Modell
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# ========================================
# 4.1 Feature Importance
# ========================================
print("4.1 Feature Importance Analysis")
print("================================")

feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nTop 10 wichtigste Features:")
print(feature_importance.head(10))

# Feature Importance Plot
plt.figure(figsize=(10, 8))
top_10_features = feature_importance.head(10)
sns.barplot(x='Importance', y='Feature', data=top_10_features)
plt.title('Top 10 wichtigste Features für das Modell')
plt.xlabel('Relative Wichtigkeit')
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.close()

# ========================================
# 4.2 Metriken
# ========================================
print("\n4.2 Geeignete Messmetrik")
print("========================")

# Vorhersagen
y_pred = rf_model.predict(X_test)

# Metriken berechnen
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"\nFür dieses Regressionsmodell sind folgende Metriken geeignet:")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Square Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"\nDie beste Metrik für dieses Modell ist der R² Score, da er zeigt,")
print(f"wie gut das Modell die Varianz in den Daten erklärt.")

# ========================================
# 4.3 Confusion Matrix
# ========================================
print("\n4.3 Confusion Matrix und Klassifikation")
print("=======================================")

# Kategorisierung
def categorize_traffic(value):
    if value < 100:
        return 'Niedrig'
    elif value < 300:
        return 'Mittel'
    else:
        return 'Hoch'

y_test_cat = y_test.apply(categorize_traffic)
y_pred_cat = pd.Series(y_pred).apply(categorize_traffic)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test_cat, y_pred_cat, labels=['Niedrig', 'Mittel', 'Hoch'])
conf_matrix_df = pd.DataFrame(conf_matrix, 
                              index=['Niedrig', 'Mittel', 'Hoch'], 
                              columns=['Niedrig', 'Mittel', 'Hoch'])

print("\nConfusion Matrix:")
print(conf_matrix_df)

# Confusion Matrix Plot
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Zugkategorien')
plt.xlabel('Vorhergesagte Kategorie')
plt.ylabel('Tatsächliche Kategorie')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.close()

# Berechnung von Metriken
precision, recall, f1, support = precision_recall_fscore_support(y_test_cat, y_pred_cat, average=None)
categories = ['Niedrig', 'Mittel', 'Hoch']

print("\nSensitivität (Recall) und Präzision für jede Kategorie:")
for i, cat in enumerate(categories):
    print(f"\n{cat}:")
    print(f"  Sensitivität (Recall): {recall[i]:.4f}")
    print(f"  Präzision: {precision[i]:.4f}")

# Spezifizität berechnen
print("\nSpezifizität für jede Kategorie:")
for i, cat in enumerate(categories):
    tn = np.sum(conf_matrix) - conf_matrix[i,:].sum() - conf_matrix[:,i].sum() + conf_matrix[i,i]
    fp = conf_matrix[:,i].sum() - conf_matrix[i,i]
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"{cat}: {specificity:.4f}")

# ========================================
# 4.4 Zusammenfassung
# ========================================
print("\n4.4 Zusammenfassung der Modellleistung")
print("======================================")

summary = f"""
Das entwickelte Random Forest Regressionsmodell zeigt eine ausgezeichnete Leistung:

Hauptmetriken:
- R² Score: {r2:.4f} (erklärt {r2*100:.1f}% der Varianz)
- Mean Absolute Error: {mae:.1f} Züge pro Tag
- RMSE: {rmse:.1f} Züge pro Tag
- MAPE: {mape:.1f}%

Gründe für die gute Leistung:
1. Die Zugzahlen zeigen starke zeitliche Muster (Monat, Jahr), die das Modell gut erfassen kann
2. Streckenspezifische Charakteristiken sind wichtige Vorhersagefaktoren  
3. Random Forest kann gut mit kategorialen Variablen umgehen
4. Der Datensatz ist gross genug für robuste Vorhersagen

Verbesserungspotenzial:
1. Integration von Wetterinformationen
2. Berücksichtigung von Feiertagen und Schulferien
3. Analyse von Wochentag-Effekten
4. Längsschnittanalyse zur besseren Trenderfassung
"""
print(summary)

# Tatsächlich vs. Vorhergesagt Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Tatsächliche Zugzahlen')
plt.ylabel('Vorhergesagte Zugzahlen')
plt.title('Tatsächlich vs. Vorhergesagt - Zugzahlen')
plt.tight_layout()
plt.savefig('actual_vs_predicted.png', dpi=300, bbox_inches='tight')
plt.close()

print("\nAnalyse abgeschlossen!")
print("\nErstellt wurden folgende Dateien:")
print("- feature_importance.png")
print("- confusion_matrix.png")
print("- actual_vs_predicted.png")
print("\nBereit für Teil 4 der Leistungsbeurteilung!")

4.1 Feature Importance Analysis

Top 10 wichtigste Features:
                                                Feature  Importance
1396           Abschnitt_Zürich HB – Zürich Langstrasse    0.064449
26       Strecke_Bezeichnung_Genève Aéroport - Lausanne    0.047000
58    Strecke_Bezeichnung_Olten Ost - Heitersberg - ...    0.046724
35    Strecke_Bezeichnung_Killwangen - Zürich Altste...    0.043938
110   Strecke_Bezeichnung_Zürich Oerlikon-Nord / Zür...    0.036875
8                              Strecke_Bezeichnung_Bern    0.027580
1401   Abschnitt_Zürich Hardbrücke – Zürich Langstrasse    0.025871
1                             Strecke_Bezeichnung_Aarau    0.025632
53       Strecke_Bezeichnung_Muttenz - HBT - Olten Nord    0.024355
1410  Abschnitt_Zürich Oerlikon – Zürich Birchsteg (...    0.022348

4.2 Geeignete Messmetrik

Für dieses Regressionsmodell sind folgende Metriken geeignet:
R² Score: 0.9951
Mean Absolute Error (MAE): 4.53
Root Mean Square Error (RMSE): 12.19
Mean Absolute Pe